<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/06_Ajuste_Fino_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ajuste Fino para Reconocimiento de Entidades Nombradas (NER)

 Este notebook muestra cómo adaptar un modelo de lenguaje pre-entrenado para identificar entidades nombradas (nombres de personas, lugares, organizaciones, etc.) en textos.

# 1) Configuración Inicial

In [2]:
# Instalamos las bibliotecas necesarias
print("Instalando bibliotecas necesarias...")
!pip install --quiet transformers datasets torch seqeval

Instalando bibliotecas necesarias...


# 2) Importar Bibliotecas

In [3]:
import torch
from transformers import (
    AutoModelForTokenClassification,  # Modelo para etiquetar tokens
    AutoTokenizer,                    # Procesador de texto
    Trainer,                          # Entrenador
    TrainingArguments,                # Configuración de entrenamiento
    DataCollatorForTokenClassification # Para procesar los datos
)
from datasets import load_dataset
import numpy as np
from seqeval.metrics import classification_report  # Para evaluar resultados de NER

# 3) Cargar Datos de Ejemplo

 Usaremos el dataset "[conll2003](https://www.clips.uantwerpen.be/conll2003/ner/)" disponible en [HuggingFace](https://huggingface.co/datasets/eriktks/conll2003), ya que es un estándar para NER.
   
   Contiene textos etiquetados con cuatro tipos de entidades:
 - PER: Personas
 - ORG: Organizaciones
 - LOC: Lugares
 - MISC: Misceláneos

In [4]:
print("\nCargando datos de ejemplo...")
dataset = load_dataset("conll2003", trust_remote_code=True)

# Mostrar información sobre el dataset
print("\nEstructura del dataset:")
print(f"Conjunto de entrenamiento: {len(dataset['train'])} ejemplos")
print(f"Conjunto de validación: {len(dataset['validation'])} ejemplos")
print(f"Conjunto de prueba: {len(dataset['test'])} ejemplos")

# Mostrar un ejemplo del dataset
print("\nEjemplo del dataset:")
ejemplo = dataset['train'][0]
print("Texto:", ' '.join(ejemplo['tokens']))
print("Etiquetas:", ejemplo['ner_tags'])


Cargando datos de ejemplo...


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]


Estructura del dataset:
Conjunto de entrenamiento: 14041 ejemplos
Conjunto de validación: 3250 ejemplos
Conjunto de prueba: 3453 ejemplos

Ejemplo del dataset:
Texto: EU rejects German call to boycott British lamb .
Etiquetas: [3, 0, 7, 0, 0, 0, 7, 0, 0]


# 4) Preparar el Modelo Base

 Usaremos BERT-cased para manejar mayúsculas y minúsculas

In [5]:
model_name = "bert-base-cased"  # Usamos 'cased' porque las mayúsculas son importantes para NER
print("\nCargando modelo y tokenizer...")

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=9  # Número de etiquetas en conll2003
)


Cargando modelo y tokenizer...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 5) Preparar los Datos

 Función para convertir texto a formato que entiende el modelo

In [6]:
def preparar_datos(ejemplos):
    """
    Prepara los datos para el entrenamiento:
    1. Tokeniza el texto (divide en subpalabras si es necesario)
    2. Alinea las etiquetas con los tokens
    3. Maneja casos especiales (tokens especiales como [CLS], [SEP], etc.)
    """
    # Tokenizar el texto (ya dividido en palabras)
    tokens = tokenizer(
        ejemplos["tokens"],
        truncation=True,        # Cortar si es muy largo
        is_split_into_words=True  # Indicar que el texto ya está dividido en palabras
    )

    # Preparar las etiquetas para cada token
    todas_etiquetas = []

    # Para cada frase en el batch
    for i in range(len(ejemplos["tokens"])):
        # Obtener los IDs que mapean tokens a palabras originales
        word_ids = tokens.word_ids(batch_index=i)
        etiquetas_actuales = []

        # Para cada token en la frase
        for word_idx in word_ids:
            if word_idx is None:
                # Token especial ([CLS], [SEP], etc.)
                etiquetas_actuales.append(-100)
            else:
                # Usar la etiqueta de la palabra original
                etiquetas_actuales.append(ejemplos["ner_tags"][i][word_idx])

        todas_etiquetas.append(etiquetas_actuales)

    tokens["labels"] = todas_etiquetas
    return tokens

print("\nPreparando datos...")
# Procesar todos los conjuntos de datos
datos_procesados = dataset.map(
    preparar_datos,
    batched=True,
    remove_columns=dataset["train"].column_names
)


Preparando datos...


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Muestra un ejemplo de cómo quedan los datos procesados

In [7]:
print("\nEjemplo de datos procesados:")
ejemplo = datos_procesados["train"][0]
print("Input IDs (tokens):", ejemplo["input_ids"][:10], "...")
print("Etiquetas:", ejemplo["labels"][:10], "...")


Ejemplo de datos procesados:
Input IDs (tokens): [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913] ...
Etiquetas: [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0] ...


# 6) Configurar el Entrenamiento

Configuración básica para el entrenamiento

In [8]:
argumentos_entrenamiento = TrainingArguments(
    output_dir="./resultados_ner",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    learning_rate=2e-5,
    load_best_model_at_end=True,
    report_to="none",
)

# 7) Entrenar el Modelo

Crear el colector de datos para procesar los batches

In [9]:
# Crear el colector de datos para procesar los batches
data_collator = DataCollatorForTokenClassification(tokenizer)

# Función para calcular métricas
def calcular_metricas(pred):
    predictions, labels = pred.predictions, pred.label_ids
    predictions = np.argmax(predictions, axis=2)

    # Remover etiquetas especiales (-100)
    true_predictions = [
        [datos_procesados["train"].features["labels"].feature.int2str(p) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [datos_procesados["train"].features["labels"].feature.int2str(l) for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Calcular métricas usando seqeval
    results = classification_report(true_labels, true_predictions, output_dict=True)
    return {
        "precision": results["micro avg"]["precision"],
        "recall": results["micro avg"]["recall"],
        "f1": results["micro avg"]["f1-score"],
    }

print("\nIniciando entrenamiento...")
trainer = Trainer(
    model=model,
    args=argumentos_entrenamiento,
    train_dataset=datos_procesados["train"],
    eval_dataset=datos_procesados["validation"],
    data_collator=data_collator,
    compute_metrics=calcular_metricas
)

# Entrenar el modelo
trainer.train()


Iniciando entrenamiento...


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# 8) Usar el Modelo

In [ ]:
def identificar_entidades(texto):
    """
    Función para identificar entidades en un texto nuevo
    """
    # Tokenizar el texto
    tokens = tokenizer(texto, return_tensors="pt", truncation=True)

    # Obtener predicciones
    with torch.no_grad():
        outputs = model(**tokens)
        predictions = torch.argmax(outputs.logits, dim=2)

    # Decodificar predicciones
    predicted_tokens = []
    for token_id, label_id in zip(tokens["input_ids"][0], predictions[0]):
        if label_id != -100:  # Ignorar tokens especiales
            token = tokenizer.decode([token_id])
            label = datos_procesados["train"].features["labels"].feature.int2str(label_id.item())
            if label != "O":  # Solo mostrar tokens con entidades
                predicted_tokens.append((token, label))

    return predicted_tokens

# Ejemplo de uso
print("\nProbando el modelo:")
texto_prueba = "John Smith works at Microsoft in New York."
entidades = identificar_entidades(texto_prueba)
print(f"\nTexto: {texto_prueba}")
print("Entidades encontradas:")
for token, label in entidades:
    print(f"- {token}: {label}")